# Inspect Various Attributes

In [1]:
import sqlite3

In [2]:
import scripts.buildSqlite

10231 entries added to table SStuBs
63923 entries added to table SStuBs Large
25539 entries added to table Bugs
153652 entries added to table Bugs Large
Created indices


In [3]:
conn = sqlite3.connect('../database/sstubs.db')
cursor = conn.cursor()

### 1. Unique SHA1

#### 1.1. Bugs

In [4]:
query = 'SELECT count(DISTINCT child), count() FROM bugs_large'
for unq, tot in cursor.execute(query):
    print(f'There are total {unq:,} unique fixCommitSHA1 among total {tot:,} entries')
    print(f'That is each commit fixes nearly {round(tot / unq, 2)} lines of bugs')

There are total 66,261 unique fixCommitSHA1 among total 153,652 entries
That is each commit fixes nearly 2.32 lines of bugs


#### 1.2. SStuBs

In [5]:
query = 'SELECT count(DISTINCT child), count() FROM sstubs_large'
for unq, tot in cursor.execute(query):
    print(f'There are total {unq:,} unique fixCommitSHA1 among total {tot:,} entries')
    print(f'That is each commit contains nearly {round(tot / unq, 2)} lines of stupid bugs')

There are total 24,486 unique fixCommitSHA1 among total 63,923 entries
That is each commit contains nearly 2.61 lines of stupid bugs


### 2. Chance of introducing new bug / skipping existing bug

If the same (line, file, project) sequence contains a `parentSHA1`
that was `childSHA1` in a previous entry, that means that line required a subsequent fix

In [6]:
query = '''SELECT count(*) FROM bugs_large WHERE (child, project, file, line) IN (
    SELECT parent, project, file, line FROM bugs_large
)'''
for row in cursor.execute(query):
    print(f'{row[0]:,} fixes required another fix')
print('Need to make it a tree to see how long this fixing goes')

847 fixes required another fix
Need to make it a tree to see how long this fixing goes


### 3. Unique Bug Entry

#### 3.1. Bugs

In [7]:
query = '''SELECT count() as num_grp, sum(cnt) as num_entry
           FROM (
                SELECT *, count(*) as cnt
                FROM bugs_large
                GROUP BY parent, child, file, line, project
                HAVING count(*) > 1
            )'''
for unq, tot in cursor.execute(query):
    print(f'In bugs, {tot:,} entries in sstubs shares the same data')
    print(f'with {unq:,} unique sequences')

In bugs, 1,100 entries in sstubs shares the same data
with 456 unique sequences


#### 3.2. SStuBs

In [8]:
query = '''SELECT count() as num_grp, sum(cnt) as num_entry
           FROM (
                SELECT *, count(*) as cnt
                FROM sstubs_large
                GROUP BY parent, child, file, line, type, project
                HAVING count(*) > 1
            )'''
for unq, tot in cursor.execute(query):
    print(f'In sstubs, {tot:,} entries in sstubs shares the same data')
    print(f'with {unq:,} unique sequences')

In sstubs, 1,680 entries in sstubs shares the same data
with 840 unique sequences


> Need to identify why the constitute different entry

### 4. Fix-commit tree

In [9]:
from collections import deque

In [10]:
roots = {}
query = 'SELECT parent, child FROM bugs_large'
for parent, child in cursor.execute(query):
    if parent not in roots:
        roots[parent] = set()
    roots[parent].add(child)

    if child not in roots:
        roots[child] = set()
    roots[child].add(parent)

print(f'{len(roots):,} unique SHA1')

126,898 unique SHA1


In [11]:
def bfs(start, return_path=False):
    visited = set()
    distance = {start: 0}
    queue = deque()
    queue.append(start)
    # mark `start` as visited
    visited.add(start)

    max_dist = 0
    distant_child = None
    # `parent_map` is used to trace back from leaf to root
    parent_map = {}
    while queue:
        a = queue.popleft()
        for b in roots[a]:
            if b not in visited:
                parent_map[b] = a
                visited.add(b)
                distance[b] = distance[a] + 1
                queue.append(b)

                if distance[b] > max_dist:
                    max_dist = distance[b]
                    distant_child = b
    if not return_path:
        return distant_child, max_dist

    path = [distant_child]
    node = distant_child
    while node in parent_map:
        node = parent_map[node]
        path.insert(0, node)
    return distant_child, max_dist, path

# just finding a commit sequence of length 5
path = []
for firstParent in roots.keys():
    longestPathStart, _ = bfs(firstParent)
    longestPathEnd, longestDistance = bfs(longestPathStart)
    if longestDistance == 5:
        _, _, path = bfs(longestPathStart, True)
        print('Path:')
        print(*path, sep='\n                   ⇓\n')
        break

Path:
a791de06c8b824094a2875ce54b934e734888f27
                   ⇓
ae85364071f63f413788dab839d6da7c06ae58fc
                   ⇓
2c7c0a297152ab15e36d477392a30f3a6b66627c
                   ⇓
29d8d6d6d0f762b15dca5db757b468875d90a1cd
                   ⇓
89eb8a7828b3b68c640f5b3f5b1db990496b1aaa
                   ⇓
e0316316807cea6fb5951874041a937855a7c730


In [12]:
parent_child = []
placeholder = []
for i in range(1, len(path)):
    parent_child.append(path[i-1])
    parent_child.append(path[i])
    placeholder.append('(parent=? AND child=?)')
placeholder = ' OR '.join(placeholder)

query = f'''SELECT parent, child, file
            FROM bugs_large
            WHERE {placeholder}
        '''
for parent, child, file in cursor.execute(query, parent_child):
    print(parent, child, file, sep='\n', end='\n\n')

a791de06c8b824094a2875ce54b934e734888f27
ae85364071f63f413788dab839d6da7c06ae58fc
src/components/org/apache/jmeter/timers/ConstantThroughputTimer.java

a791de06c8b824094a2875ce54b934e734888f27
ae85364071f63f413788dab839d6da7c06ae58fc
src/components/org/apache/jmeter/timers/PoissonRandomTimer.java

a791de06c8b824094a2875ce54b934e734888f27
ae85364071f63f413788dab839d6da7c06ae58fc
src/components/org/apache/jmeter/visualizers/Graph.java

a791de06c8b824094a2875ce54b934e734888f27
ae85364071f63f413788dab839d6da7c06ae58fc
src/components/org/apache/jmeter/visualizers/Graph.java

a791de06c8b824094a2875ce54b934e734888f27
ae85364071f63f413788dab839d6da7c06ae58fc
src/components/org/apache/jmeter/visualizers/Graph.java

a791de06c8b824094a2875ce54b934e734888f27
ae85364071f63f413788dab839d6da7c06ae58fc
src/components/org/apache/jmeter/visualizers/Graph.java

a791de06c8b824094a2875ce54b934e734888f27
ae85364071f63f413788dab839d6da7c06ae58fc
src/components/org/apache/jmeter/visualizers/RespTimeGraphVisua

> To summarize, building the tree did not give us anything valuable.
> Having a sequence of SHA1 is literally worthless.

### 5. Number of Owner and Projects

In [13]:
query = '''SELECT count(DISTINCT project) FROM bugs_large'''
for row in cursor.execute(query):
    print(f'There are {row[0]} projects / repo')

There are 759 projects / repo


In [14]:
query = '''SELECT count(DISTINCT substr(project, 0, instr(project, '.'))) FROM bugs_large'''
for row in cursor.execute(query):
    print(f'There are {row[0]} repo owners')

There are 558 repo owners


In [15]:
query = '''
SELECT count()
FROM (
    SELECT *
    FROM (
        SELECT
           substr(project, 0, instr(project, '.')) as owner,
           substr(project, instr(project, '.') + 1) as repo
        FROM bugs_large) AS owners
    GROUP BY owner
    HAVING count(DISTINCT repo) > 1
)'''
for row in cursor.execute(query):
    print(f'There are {row[0]} repo owners having multiple projects / repo')

There are 57 repo owners having multiple projects / repo


In [16]:
query = '''
SELECT count()
FROM (
    SELECT DISTINCT owner
    FROM (
        SELECT
           substr(project, 0, instr(project, '.')) as owner,
           substr(project, instr(project, '.') + 1) as repo
        FROM bugs_large) AS owners
    WHERE owner = repo
)'''
for row in cursor.execute(query):
    print(f'There are {row[0]} repo having the same name as owner')

There are 65 repo having the same name as owner


### 6. Sources That Required Sub-Sequent Changes

In [17]:
import pandas as pd
pd.set_option('display.max_rows', 0)

In [18]:
query = '''
SELECT a.before as first, a.after as second, b.after as third
FROM sstubs AS a
    INNER JOIN sstubs as b on a.after = b.before
WHERE 5 < length(b.before) AND length(b.before) < 12
'''
pd.DataFrame(cursor.execute(query), columns=['First', 'Second', 'Third'])

,First,Second,Third
0,IOException e,Exception e,Throwable e
1,NumberFormatException nfe,Exception e,Throwable e
2,IllegalArgumentException e,Exception e,Throwable e
3,IllegalArgumentException e,Exception e,Throwable e
4,ReflectiveOperationException e,Exception e,Throwable e
5,ReflectiveOperationException e,Exception e,Throwable e
6,IOException e,Exception e,Throwable e
7,IOException e,Exception e,Throwable e
8,IOException e,Exception e,Throwable e
...,...,...,...


### 7. Length of SHA1 to Uniquely identify a commit

In [19]:
for table in ['bugs', 'bugs_large', 'sstubs', 'sstubs_large']:
    query = f'''
    SELECT count()
    FROM (
         SELECT parent FROM {table} UNION SELECT child FROM {table}
    )
    '''
    numUnqCommits = next(cursor.execute(query))
    for i in range(40):
        query = f'''
        SELECT count(DISTINCT substr(parent, 0, {i}))
        FROM (
             SELECT parent FROM {table} UNION SELECT child FROM {table}
        )
        '''
        numIdentifiableCommits = next(cursor.execute(query))
        if numUnqCommits == numIdentifiableCommits:
            print(f'For {table} {i} chars are enough to identify a commit SHA1')
            break

For bugs 9 chars are enough to identify a commit SHA1
For bugs_large 10 chars are enough to identify a commit SHA1
For sstubs 8 chars are enough to identify a commit SHA1
For sstubs_large 9 chars are enough to identify a commit SHA1
